# Exercices

## **Préliminaires**: Clone de votre repo et imports

In [19]:
! git clone https://github.com/nouhalahyen/exam_2025.git
! cp exam_2025/utils/utils_exercices.py .

import copy
import numpy as np
import torch.nn as nn
import torch
from utils_exercices import Dataset1

fatal: destination path 'exam_2025' already exists and is not an empty directory.


**Clef personnelle pour la partie théorique**

Dans la cellule suivante, choisir un entier entre 100 et 1000 (il doit être personnel). Cet entier servira de graine au générateur de nombres aléatoire a conserver pour tous les exercices.



In [20]:
mySeed = 300

\

---

\

\

**Exercice 1** *Une relation linéaire*

La fonction *generate_dataset* fournit deux jeux de données (entraînement et test). Pour chaque jeu de données, la clef 'inputs' donne accès à un tableau numpy (numpy array) de prédicteurs empilés horizontalement : chaque ligne $i$ contient trois prédicteurs $x_i$, $y_i$ et $z_i$. La clef 'targets' renvoie le vecteur des cibles $t_i$. \

Les cibles sont liées aux prédicteurs par le modèle:
$$ t = \theta_0 + \theta_1 x + \theta_2 y + \theta_3 z + \epsilon$$ où $\epsilon \sim \mathcal{N}(0,\eta)$


In [21]:
from utils_exercices import generate_dataset, Dataset1
train_set, test_set = generate_dataset(mySeed)

**Q1** Par quelle méthode simple peut-on estimer les coefficients $\theta_k$ ? La mettre en oeuvre avec la librairie python de votre choix.

Pour déterminer les coefficients $\theta$, on applique la méthode des moindres carrés, qui consiste à minimiser la somme des carrés des écarts :

$$
\min ||X\theta - y||^2
$$

où $X$ représente la matrice des variables prédictives et $y$ est le vecteur des observations. La solution analytique s'écrit comme suit :

$$
\theta = (X^T X)^{-1} X^T y
$$

In [22]:
from numpy.linalg import inv

# Données d'entraînement
X = train_set['inputs']
y = train_set['targets']

# Ajout d'une colonne de biais (1)
X_b = np.hstack([X, np.ones((X.shape[0], 1))])

# Calcul de \theta
theta_hat = inv(X_b.T @ X_b) @ X_b.T @ y
print("Coefficients estimés (theta):", theta_hat)


Coefficients estimés (theta): [ 2.97230274  2.9365536   5.85815285 15.01770145]


**Q2** Dans les cellules suivantes, on se propose d'estimer les $\theta_k$ grâce à un réseau de neurones entraîné par SGD. Quelle architecture s'y prête ? Justifier en termes d'expressivité et de performances en généralisation puis la coder dans la cellule suivante.

Pour estimer les coefficients $\theta_k$, une architecture simple, comme un réseau de neurones avec une seule couche linéaire entièrement connectée, est bien adaptée. Cette configuration est suffisante pour capturer une relation linéaire, tout en réduisant le risque de sur-apprentissage et en maintenant une capacité d'apprentissage efficace sur les données.


In [23]:
# Dataset et dataloader :
dataset = Dataset1(train_set['inputs'], train_set['targets'])
dataloader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=True)

# A coder :
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc = nn.Linear(3, 1)

    def forward(self, x):
        return self.fc(x)

**Q3** Entraîner cette architecture à la tâche de régression définie par les entrées et sorties du jeu d'entraînement (compléter la cellule ci-dessous).

In [24]:
# Initialize model, loss, and optimizer
mySimpleNet = SimpleNet()
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(mySimpleNet.parameters(), lr=0.01)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    for batch_inputs, batch_targets in dataloader:
        # Forward pass : calcul des prédictions
        predictions = mySimpleNet(batch_inputs)

        # Calcul de la perte
        loss = criterion(predictions, batch_targets)

        # Backward pass : calcul du gradient
        optimizer.zero_grad()
        loss.backward()

        # Mise à jour des poids
        optimizer.step()

    # Afficher la perte toutes les 50 époques
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 50/500, Loss: 24.8701
Epoch 100/500, Loss: 30.8892
Epoch 150/500, Loss: 26.8858
Epoch 200/500, Loss: 27.9139
Epoch 250/500, Loss: 43.0794
Epoch 300/500, Loss: 32.3304
Epoch 350/500, Loss: 30.7474
Epoch 400/500, Loss: 28.7571
Epoch 450/500, Loss: 33.6057
Epoch 500/500, Loss: 34.9630


**Q4** Où sont alors stockées les estimations des  $\theta_k$ ? Les extraire du réseau *mySimpleNet* dans la cellule suivante.

Les coefficients $\theta_k$ estimés sont enregistrés dans les paramètres de la couche linéaire du réseau, à savoir :  
- Les poids  de la couche linéaire.  
- Le biais  de la couche linéaire.


In [26]:
weights = mySimpleNet.fc.weight.data.numpy()
bias = mySimpleNet.fc.bias.data.numpy()

print("Coefficients estimés (poids) :", weights)
print("Biais estimé :", bias)

Coefficients estimés (poids) : [[0.01707954 0.01955457 0.05218352]]
Biais estimé : [17.971333]


**Q5** Tester ces estimations sur le jeu de test et comparer avec celles de la question 1. Commentez.

In [27]:
# prompt: Q5 Tester les estimations sur le jeu de test et les comparer avec celles de la question 1. Fournir une analyse.

# Prédictions sur le jeu de test avec les coefficients estimés du modèle linéaire (Q1)
X_test = test_set['inputs']
y_test = test_set['targets']
X_test_with_bias = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
y_pred_linear = X_test_with_bias @ theta_hat

# Prédictions sur le jeu de test avec le réseau de neurones entraîné (Q4)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_pred_nn = mySimpleNet(X_test_tensor).detach().numpy().flatten()

# Calcul des erreurs quadratiques moyennes (MSE) pour chaque méthode
mse_linear = np.mean((y_test - y_pred_linear)**2)
mse_nn = np.mean((y_test - y_pred_nn)**2)

# Affichage des résultats
print(f"MSE (Modèle linéaire - Question 1) : {mse_linear}")
print(f"MSE (Réseau de neurones - Question 4) : {mse_nn}")

# Comparaison des performances
print("Analyse des performances :")
if mse_linear < mse_nn:
    print("Le modèle linéaire (Q1) offre de meilleures performances sur le jeu de test.")
elif mse_linear > mse_nn:
    print("Le réseau de neurones (Q4) offre de meilleures performances sur le jeu de test.")
else:
    print("Les deux méthodes présentent des performances équivalentes sur le jeu de test.")


MSE (Modèle linéaire - Question 1) : 4.09572233590508
MSE (Réseau de neurones - Question 4) : 32.9152994601767
Analyse des performances :
Le modèle linéaire (Q1) offre de meilleures performances sur le jeu de test.


\

---

\

**Exercice 2** *Champ réceptif et prédiction causale*

Le réseau défini dans la cellule suivante est utilisé pour faire le lien entre les valeurs $(x_{t' \leq t})$ d'une série temporelle d'entrée et la valeur présente $y_t$ d'une série temporelle cible.

In [29]:
import torch.nn as nn
import torch.nn.functional as F
from utils_exercices import Outconv, Up_causal, Down_causal

class Double_conv_causal(nn.Module):
    '''(conv => BN => ReLU) * 2, with causal convolutions that preserve input size'''
    def __init__(self, in_ch, out_ch, kernel_size=3, dilation=1):
        super(Double_conv_causal, self).__init__()
        self.kernel_size = kernel_size
        self.dilation = dilation
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn1 = nn.BatchNorm1d(out_ch)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=kernel_size, padding=0, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_ch)

    def forward(self, x):
        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = F.pad(x, ((self.kernel_size - 1) * self.dilation, 0))
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        return x


class causalFCN(nn.Module):
    def __init__(self, dilation=1):
        super(causalFCN, self).__init__()
        size = 64
        n_channels = 1
        n_classes = 1
        self.inc = Double_conv_causal(n_channels, size)
        self.down1 = Down_causal(size, 2*size)
        self.down2 = Down_causal(2*size, 4*size)
        self.down3 = Down_causal(4*size, 8*size, pooling_kernel_size=5, pooling_stride=5)
        self.down4 = Down_causal(8*size, 4*size, pooling=False, dilation=2)
        self.up2 = Up_causal(4*size, 2*size, kernel_size=5, stride=5)
        self.up3 = Up_causal(2*size, size)
        self.up4 = Up_causal(size, size)
        self.outc = Outconv(size, n_classes)
        self.n_classes = n_classes

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up2(x5, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Exemple d'utilisation
model = causalFCN()
# Série temporelle d'entrée (x_t):
input_tensor1 = torch.rand(1, 1, 10000)
# Série temporelle en sortie f(x_t):
output = model(input_tensor1)
print(output.shape)

torch.Size([1, 1, 10000])


**Q1** De quel type de réseau de neurones s'agit-il ? Combien de paramètres la couche self.Down1 compte-t-elle (à faire à la main) ?
Combien de paramètres le réseau entier compte-t-il (avec un peu de code) ?

Le réseau est un Fully Convolutional Network (FCN) avec des convolutions causales. Ces convolutions assurent que chaque sortie $y_t$ ne dépend que des entrées $x_{t'}$ pour lesquelles $t' \leq t$, respectant ainsi la causalité dans les séries temporelles. Cela en fait un modèle particulièrement adapté aux contextes séquentiels pour la prédiction des valeurs cibles.

Calcul du nombre de paramètres pour la couche self.down1 :

    Formule pour une couche Conv1d :
    Nombre de parameˋtres=(in_ch×out_ch×kernel_size)+out_chNombre de parameˋtres=(in_ch×out_ch×kernel_size)+out_ch

    Formule pour une couche BatchNorm1d :
    Nombre de parameˋtres=2×out_chNombre de parameˋtres=2×out_ch

Pour self.down1 :

    Canaux d'entrée : $in_ch = 64$, canaux de sortie : $out_ch = 128$, taille du noyau : $kernel_size = 3$
    Conv1d:(64×128×3)+128=24672Conv1d:(64×128×3)+128=24672
    BatchNorm1d:2×128=256BatchNorm1d:2×128=256
    Nombre total :
    24672+256=24928 parameˋtres24672+256=24928 parameˋtres

Calcul du nombre total de paramètres du réseau :

En additionnant les paramètres de toutes les couches, le réseau contient :
Nombre total de parameˋtres=2872641


In [30]:

print(model.down1)
# parameters self.down1
for name, param in model.down1.named_parameters():
    print(f"Layer: {name}, Parameters: {param.numel()}")

# number total of parameters:
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Nombre total de paramètres : {total_params}")

Down_causal(
  (mpconv): Sequential(
    (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Double_conv_causal(
      (conv1): Conv1d(64, 128, kernel_size=(3,), stride=(1,))
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)
Layer: mpconv.1.conv1.weight, Parameters: 24576
Layer: mpconv.1.conv1.bias, Parameters: 128
Layer: mpconv.1.bn1.weight, Parameters: 128
Layer: mpconv.1.bn1.bias, Parameters: 128
Layer: mpconv.1.conv2.weight, Parameters: 49152
Layer: mpconv.1.conv2.bias, Parameters: 128
Layer: mpconv.1.bn2.weight, Parameters: 128
Layer: mpconv.1.bn2.bias, Parameters: 128
Nombre total de paramètres : 2872641


**Q2** Par quels mécanismes la taille du vecteur d'entrée est-elle réduite ? Comment est-elle restituée dans la deuxième partie du réseau ?

### Réduction de la dimension du vecteur d'entrée

Dans la première partie du réseau, la taille du vecteur d'entrée est réduite grâce aux opérations suivantes :

#### **1. Pooling dans les couches `Down_causal`**  
- Les couches de pooling, comme `MaxPool1d`, diminuent la longueur de la séquence en sous-échantillonnant les données.  
- Par exemple, un `MaxPool1d` avec un `kernel_size=2` et un `stride=2` réduit la longueur de la séquence de moitié.

---

### Restauration de la dimension dans la seconde partie du réseau

La taille du vecteur est progressivement restaurée dans la deuxième partie du réseau via les mécanismes suivants :  

#### **1. Upsampling dans les couches `Up_causal`**  
- Ces couches utilisent des convolutions transposées ou des techniques similaires pour augmenter la longueur du vecteur.  
- Par exemple, une couche `Up_causal` avec un `kernel_size=5` et un `stride=5` multiplie la longueur de la séquence par 5.  

#### **2. Concatenation avec les sorties intermédiaires**  
- Lors de l'upsampling, les sorties intermédiaires des couches descendantes (`down1`, `down2`, etc.) sont concaténées avec les sorties des couches montantes correspondantes (`up2`, `up3`, etc.).  
- Cette opération permet de récupérer les informations perdues pendant la réduction de la taille et améliore la précision globale du modèle.  


**Q3** Par quels mécanismes le champ réceptif est-il augmenté ? Préciser par un calcul la taille du champ réceptif en sortie de *self.inc*.

## Méthodes pour augmenter le champ réceptif

Le champ réceptif est élargi grâce aux mécanismes suivants :

### 1. Convolutions avec des kernels de grande taille  
Chaque couche de convolution capture les informations sur plusieurs pas de temps, ce qui élargit le champ réceptif.

### 2. Convolutions dilatées (dilated convolutions)  
En insérant des "espaces" entre les éléments pris en compte par le kernel, la dilatation permet de couvrir une plage plus large sans augmenter le nombre de paramètres.

### 3. Empilement de couches convolutionnelles  
Les couches successives augmentent progressivement le champ réceptif. Chaque couche dépend des sorties de la précédente, qui intègrent déjà des informations sur une plage étendue.

---

## Calcul du champ réceptif de `self.inc`

La couche `self.inc` est composée de deux convolutions successives : `conv1` et `conv2`, avec les caractéristiques suivantes :  
- **Taille du kernel** : `kernel_size = 3`  
- **Dilatation** : `dilation = 1`  

### Formule générale pour le calcul du champ réceptif  
\[
\text{Champ réceptif} = (\text{kernel\_size} - 1) \times (\text{dilation} + 1) + 1
\]

### Étapes de calcul :

#### 1. Champ réceptif de `conv1`  
\[
\text{Champ\_réceptif\_conv1} = (3 - 1) \times (1 + 1) + 1 = 3
\]

#### 2. Champ réceptif de `conv2`  
Le champ réceptif de `conv2` prend en compte celui de `conv1` :  
\[
\text{Champ\_réceptif\_conv2} = (\text{Champ\_réceptif\_conv1} - 1) + ((3 - 1) \times (1 + 1) + 1)
\]  
\[
\text{Champ\_réceptif\_conv2} = (3 - 1) + 3 = 5
\]

---

## Résultat final  
Le champ réceptif total de la couche `self.inc` est :  
\[
\textbf{5}
\]



**Q4** Par un bout de code, déterminer empiriquement la taille du champ réceptif associé à la composante $y_{5000}$ du vecteur de sortie. (Indice: considérer les sorties associées à deux inputs qui ne diffèrent que par une composante...)

**Q5** $y_{5000}$ dépend-elle des composantes $x_{t, \space t > 5000}$ ? Justifier de manière empirique puis préciser la partie du code de Double_conv_causal qui garantit cette propriété de "causalité" en justifiant.  



\

---

\

\

Exercice 3: "Ranknet loss"

Un [article récent](https://https://arxiv.org/abs/2403.14144) revient sur les progrès en matière de learning to rank. En voilà un extrait :


<img src="https://raw.githubusercontent.com/nanopiero/exam_2025/refs/heads/main/utils/png_exercice3.PNG?token=GHSAT0AAAAAAC427DACOPGNDNN6UDOLVLLAZ4BB2JQ" alt="extrait d'un article" width="800">

**Q1** Qu'est-ce que les auteurs appellent "positive samples" et "negative samples" ? Donner un exemple.

**Q2** Dans l'expression de $\mathcal{L}_{RankNet}$, d'où proviennent les $z_i$ ? Que représentent-ils ?  

**Q3** Pourquoi cette expression conduit-elle à ce que, après apprentissage, "the estimated
value of positive samples is greater than that of negative samples
for each pair of positive/negative samples" ?

**Q4** Dans le cadre d'une approche par deep learning, quels termes utilise-t-on pour qualifier les réseaux de neurones exploités et la modalité suivant laquelle ils sont entraînés ?